## For n=2 the code and the output
__results__ :
 total number of depnedent funtions=0   
 total super dependent s_boxes found=0

In [1]:
# Function to generate outputs for the Boolean function f with coefficients c0 to c6
# for n=2
def is_super_dependent(L):
    """
    Check if the output vector L is super-dependent on all variables x1, x2, and x3.
    """
    # Check if flipping x1 changes the outputs
    change_x1 = (L[0] != L[2])  # Compare outputs for (00 <-> 10)
    change_x2= (L[1] != L[3])  # Compare outputs for (01 <-> 11)
    # Check if flipping x2 changes the outputs
    change_x3 = (L[0] != L[1])  # Compare outputs for (00 <-> 01)
    # Check if flipping x3 changes the outputs
    change_x4 = (L[2] != L[3])  # Compare outputs for (10 <-> 11)
   

    # A function is super-dependent if it changes on flipping any of x1, x2, or x3
    return change_x1 and change_x2 and change_x3 and change_x4
def fn(c0, c1, c2, c3):
    L = []
    # Iterate over all possible values of x1, x2, x3 (0 or 1)
    for x1 in range(0, 2):
        for x2 in range(0, 2):
            for x3 in range(0, 2):
                # Define the Boolean function in ANF (Algebraic Normal Form)
                f = c0 ^ (c1 & x1) ^ (c2 & x2)  ^ (c3 & x1 & x2) 
                # Append the result to the list
                
                L.append(f)
    return L

# Function to check the uniqueness of the vectors in matrix M


# Main loop to generate all combinations of coefficients c0 to c6
M = []  # Initialize the matrix to store the results
p=0
for c0 in range(0, 2):
    for c1 in range(0, 2):
        for c2 in range(0, 2):
            for c3 in range(0, 2):
                # Generate the vector for the Boolean function
                L = fn(c0, c1, c2, c3)
                # Add the vector to matrix M
                M.append(L)
                if(is_super_dependent(L)):
                    p=p+1
                    # Print the function coefficients and its vector output
                    print(f"{c0}{c1}{c2}{c3} : {L}")
                            

# Check for uniqueness of the rows (vectors) in the matrix M

print(p)


0


## for n=3
__result__: total s_boxes found=24576

In [ ]:
import numpy as np

def is_dependent_on_all_variables(L):
    """
    Check if the output vector L is dependent on all variables x1, x2, and x3.
    """
    change_x1 = any(L[i] != L[i ^ 4] for i in range(8))  # Compare outputs for (0xxxx <-> 1xxxx)
    change_x2 = any(L[i] != L[i ^ 2] for i in range(8))  # Compare outputs for (xx0xx <-> xx1xx)
    change_x3 = any(L[i] != L[i ^ 1] for i in range(8))  # Compare outputs for (xxx0x <-> xxx1x)
    
    return change_x1 and change_x2 and change_x3

def fn(c0, c1, c2, c3, c4, c5, c6, c7):
    L = []
    for x1 in range(2):
        for x2 in range(2):
            for x3 in range(2):
                f = (c0 ^ (c1 & x1) ^ (c2 & x2) ^ 
                     (c3 & x3) ^ (c4 & x1 & x2) ^ 
                     (c5 & x1 & x3) ^ (c6 & x2 & x3) ^ 
                     (c7 & x1 & x2 & x3))
                L.append(f)
    return L

# Generate all possible coefficients and count dependent functions
dependent_count = 0
dependent_list = []
for c0 in range(2):
    for c1 in range(2):
        for c2 in range(2):
            for c3 in range(2):
                for c4 in range(2):
                    for c5 in range(2):
                        for c6 in range(2):
                            for c7 in range(2):
                                L = fn(c0, c1, c2, c3, c4, c5, c6, c7)
                                if is_dependent_on_all_variables(L):
                                    dependent_count += 1
                                    dependent_list.append(L)
                                    # print(f"{c0}{c1}{c2}{c3}{c4}{c5}{c6}{c7} : {L}")

print(f"\nNumber of functions dependent on all variables: {dependent_count}")

# Check for permutations
permutation = 0
final_s_boxes = []
print("Starting permutation checks...")
for i in dependent_list:
    for j in dependent_list:
        for jj in dependent_list:
            num_output = []
            flag = True
            for x1 in range(2):
                for x2 in range(2):
                    for x3 in range(2):
                        idx = 4 * x1 + 2 * x2 + x3
                        if idx < len(i) and idx < len(j) and idx < len(jj):
                            y1 = i[idx]
                            y2 = j[idx]
                            y3 = jj[idx]
                            num_output.append(y1 * 4 + y2 * 2 + y3)
                        else:
                            flag = False
                            break
                if not flag:
                    break
            if flag:
                sorted_output = sorted(num_output)
                if sorted_output == list(range(8)):
                    final_s_boxes.append(num_output.copy())
                    permutation += 1

print(f"\nNumber of final S-boxes: {len(final_s_boxes)}")
for s_box in final_s_boxes:
    print(s_box)
print(f"Number of permutations: {permutation}")

# Functions to compute LAT and DDT
def compute_lat(sbox):
    n = len(sbox)
    lat = np.zeros((n, n), dtype=int)

    for input_mask in range(n):
        for output_mask in range(n):
            count = sum(1 for x in range(n) if 
                        (bin(input_mask & x).count('1') % 2) == 
                        (bin(output_mask & sbox[x]).count('1') % 2))
            lat[input_mask][output_mask] = count - (n // 2)  # Centering the values

    return lat

def compute_ddt(sbox):
    n = len(sbox)
    ddt = np.zeros((n, n), dtype=int)

    for input_diff in range(n):
        for x in range(n):
            output_diff = sbox[x] ^ sbox[x ^ input_diff]
            ddt[input_diff][output_diff] += 1

    return ddt

# Functions to compute nonlinearity and differential uniformity
def compute_nonlinearity(sbox):
    lat = compute_lat(sbox)
    n = len(sbox)
    max_value = max(max(row) for row in lat)
    nonlinearity = (n // 2) - (max_value // 2)
    return nonlinearity

def compute_differential_uniformity(sbox):
    ddt = compute_ddt(sbox)
    return max(max(row) for row in ddt)

# Process each final S-box for LAT, DDT, nonlinearity, and differential uniformity
best_s_boxes = []
best_nonlinearity = -1
best_ddt = float('inf')

for idx, sbox in enumerate(final_s_boxes):
    lat = compute_lat(sbox)
    ddt = compute_ddt(sbox)
    nonlinearity = compute_nonlinearity(sbox)
    diff_uniformity = compute_differential_uniformity(sbox)

    # Print the results
    print(f"\nS-box {idx + 1}: {sbox}")
    print("Linear Approximation Table (LAT):")
    print(lat)
    print("Difference Distribution Table (DDT):")
    print(ddt)
    print(f"Nonlinearity: {nonlinearity}")
    print(f"Differential Uniformity: {diff_uniformity}")

    # Determine if this S-box is the best one based on nonlinearity and differential uniformity
    if (nonlinearity > best_nonlinearity or 
        (nonlinearity == best_nonlinearity and diff_uniformity < best_ddt)):
        best_nonlinearity = nonlinearity
        best_ddt = diff_uniformity
        best_s_boxes = [sbox]
    elif nonlinearity == best_nonlinearity and diff_uniformity == best_ddt:
        best_s_boxes.append(sbox)

# Print the best S-boxes found
print(f"\nBest S-box(es) based on Nonlinearity and Differential Uniformity:")
for idx, s_box in enumerate(best_s_boxes):
    print(f"S-box {idx + 1}: {s_box} with Nonlinearity: {best_nonlinearity} and Differential Uniformity: {best_ddt}")



Number of functions dependent on all variables: 218
Starting permutation checks...


## for n=4


In [ ]:
import numpy as np

def is_dependent_on_all_variables(L):
    """
    Check if the output vector L is dependent on all variables x1, x2, x3, and x4.
    """
    change_x1 = any(L[i] != L[i ^ 8] for i in range(16))  # Compare outputs for (0xxxx <-> 1xxxx)
    change_x2 = any(L[i] != L[i ^ 4] for i in range(16))  # Compare outputs for (xx0xx <-> xx1xx)
    change_x3 = any(L[i] != L[i ^ 2] for i in range(16))  # Compare outputs for (xxx0x <-> xxx1x)
    change_x4 = any(L[i] != L[i ^ 1] for i in range(16))  # Compare outputs for (xxxx0 <-> xxxx1)
    
    return change_x1 and change_x2 and change_x3 and change_x4

def fn(c0, c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13, c14, c15):
    L = []
    for x1 in range(2):
        for x2 in range(2):
            for x3 in range(2):
                for x4 in range(2):
                    f = (c0 ^ (c1 & x1) ^ (c2 & x2) ^ 
                             (c3 & x3) ^ (c4 & x4) ^ 
                             (c5 & x1 & x2) ^ (c6 & x1 & x3) ^ 
                             (c7 & x1 & x4) ^ (c8 & x2 & x3) ^ 
                             (c9 & x2 & x4) ^ (c10 & x3 & x4) ^ 
                             (c11 & x1 & x2 & x3) ^ (c12 & x1 & x2 & x4) ^ 
                             (c13 & x1 & x3 & x4) ^ (c14 & x2 & x3 & x4) ^ 
                             (c15 & x1 & x2 & x3 & x4))
                    L.append(f)
    return L

# Generate all possible coefficients and count dependent functions
dependent_count = 0
dependent_list = []
for c0 in range(2):
    for c1 in range(2):
        for c2 in range(2):
            for c3 in range(2):
                for c4 in range(2):
                    for c5 in range(2):
                        for c6 in range(2):
                            for c7 in range(2):
                                for c8 in range(2):
                                    for c9 in range(2):
                                        for c10 in range(2):
                                            for c11 in range(2):
                                                for c12 in range(2):
                                                    for c13 in range(2):
                                                        for c14 in range(2):
                                                            for c15 in range(2):
                                                                L = fn(c0, c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13, c14, c15)
                                                                if is_dependent_on_all_variables(L):
                                                                    dependent_count += 1
                                                                    dependent_list.append(L)

print(f"\nNumber of functions dependent on all variables: {dependent_count}")

# Check for permutations
permutation = 0
final_s_boxes = []
print("Starting permutation checks...")
for i in dependent_list:
    for j in dependent_list:
        for jj in dependent_list:
            for kk in dependent_list:
                num_output = []
                flag = True
                for x1 in range(2):
                    for x2 in range(2):
                        for x3 in range(2):
                            for x4 in range(2):
                                idx = 8 * x1 + 4 * x2 + 2 * x3 + x4
                                if idx < len(i) and idx < len(j) and idx < len(jj) and idx < len(kk):
                                    y1 = i[idx]
                                    y2 = j[idx]
                                    y3 = jj[idx]
                                    y4 = kk[idx]
                                    num_output.append(y1 * 8 + y2 * 4 + y3 * 2 + y4)
                                else:
                                    flag = False
                                    break
                    if not flag:
                        break
                if flag:
                    sorted_output = sorted(num_output)
                    if sorted_output == list(range(16)):
                        final_s_boxes.append(num_output.copy())
                        permutation += 1

print(f"\nNumber of final S-boxes: {len(final_s_boxes)}")
for s_box in final_s_boxes:
    print(s_box)
print(f"Number of permutations: {permutation}")

# Functions to compute LAT and DDT
def compute_lat(sbox):
    n = len(sbox)
    lat = np.zeros((n, n), dtype=int)

    for input_mask in range(n):
        for output_mask in range(n):
            count = sum(1 for x in range(n) if 
                        (bin(input_mask & x).count('1') % 2) == 
                        (bin(output_mask & sbox[x]).count('1') % 2))
            lat[input_mask][output_mask] = count - (n // 2)  # Centering the values

    return lat

def compute_ddt(sbox):
    n = len(sbox)
    ddt = np.zeros((n, n), dtype=int)

    for input_diff in range(n):
        for x in range(n):
            output_diff = sbox[x] ^ sbox[x ^ input_diff]
            ddt[input_diff][output_diff] += 1

    return ddt

# Functions to compute nonlinearity and differential uniformity
def compute_nonlinearity(sbox):
    lat = compute_lat(sbox)
    n = len(sbox)
    max_value = max(max(row) for row in lat)
    nonlinearity = (n // 2) - (max_value // 2)
    return nonlinearity

def compute_differential_uniformity(sbox):
    ddt = compute_ddt(sbox)
    return max(max(row) for row in ddt)

# Process each final S-box for LAT, DDT, nonlinearity, and differential uniformity
best_s_boxes = []
best_nonlinearity = -1
best_ddt = float('inf')

for idx, sbox in enumerate(final_s_boxes):
    lat = compute_lat(sbox)
    ddt = compute_ddt(sbox)
    nonlinearity = compute_nonlinearity(sbox)
    diff_uniformity = compute_differential_uniformity(sbox)

    # Print the results
    print(f"\nS-box {idx + 1}: {sbox}")
    print("Linear Approximation Table (LAT):")
    print(lat)
    print("Difference Distribution Table (DDT):")
    print(ddt)
    print(f"Nonlinearity: {nonlinearity}")
    print(f"Differential Uniformity: {diff_uniformity}")

    # Determine if this S-box is the best one based on nonlinearity and differential uniformity
    if (nonlinearity > best_nonlinearity or 
        (nonlinearity == best_nonlinearity and diff_uniformity < best_ddt)):
        best_nonlinearity = nonlinearity
        best_ddt = diff_uniformity
        best_s_boxes = [sbox]
    elif nonlinearity == best_nonlinearity and diff_uniformity == best_ddt:
        best_s_boxes.append(sbox)

# Print the best S-boxes found
print(f"\nBest S-box(es) based on Nonlinearity and Differential Uniformity:")
for idx, s_box in enumerate(best_s_boxes):
    print(f"S-box {idx + 1}: {s_box} with Nonlinearity: {best_nonlinearity} and Differential Uniformity: {best_ddt}")
